# Day 4: Passport Processing

[Brief](https://adventofcode.com/2020/day/4)

In [14]:
import re

def parse_passports(contents):
    # split where there is an empty line
    passports = contents.split("\n\n")
    # remove replace newlines from the middle of passports
    return [p.replace("\n", " ") for p in passports]

In [58]:
def valid_passport(passport, required_fields=["byr", "iyr", "eyr", "hgt", "hcl", "ecl", "pid"]): 
    # list of fields that are required that haven't been found yet
    fields_left = required_fields.copy()
    fields = passport.split(" ")
    
    for field in fields:
        # split at colon and get first item to get the name of the field
        field_name = field.split(":")[0]
        # remove the field from the required fields list as we have found it
        if field_name in fields_left:
            fields_left.remove(field_name)
    
    # if there are more than 0 fields left, it is not valid
    return len(fields_left) <= 0

In [91]:
def count_valid(passports, func=valid_passport):
    valid = 0
    for passport in passports:
        if func(passport):
            valid += 1
    return valid

## Example

Making sure that the function works with the example file.

In [92]:
with open("example.txt", "r") as file:
    lines = file.read()

valid = count_valid(parse_passports(lines))
print("There are {} valid passports".format(valid))
assert valid == 2

There are 2 valid passports


## Part 1

In [93]:
with open("input.txt", "r") as file:
    lines = file.read()

valid = count_valid(parse_passports(lines))
print("There are {} valid passports".format(valid))

There are 239 valid passports


## Part 2

The second part requires the validation function to be slightly rewritten. I could have changed the function above but I think it's nice to see how it has been edited to work with the second half of the challenge.

### Field validation

- `byr` must be between `1920` and `2002`
- `iyr` must be between `2010` and `2020`
- `eyr` must be between `2020` and `2030`
- `hgt` must be a number followed by `cm` or `in`
    - if `cm`, the number must be between `150` and `193`
    - if `in`, the number must be between `59` and `76`
- `hcl` must be a hex color with `#` in front
- `ecl` must be `amb`, `blu`, `brn`, `gry`, `grn`, `hzl` or `oth`
- `pid` must be a 9 digit number
- `cid` can be ignored

In [123]:
def validate_field(field_name, field_value):
    if field_name == "byr":
        return int(field_value) >= 1920 and int(field_value) <= 2002
    if field_name == "iyr":
        return int(field_value) >= 2010 and int(field_value) <= 2020
    if field_name == "eyr":
        return int(field_value) >= 2020 and int(field_value) <= 2030
    if field_name == "hgt":
        match = re.match(r"^([0-9]{2,3})(cm|in)$", field_value, re.I)
        if not match:
            return False
        parts = match.groups()
        if parts[1] == "cm":
            return int(parts[0]) >= 150 and int(parts[0]) <= 193
        if parts[1] == "in":
            return int(parts[0]) >= 59 and int(parts[0]) <= 76
    if field_name == "hcl":
        return re.match(r"^#[0-9a-f]{6}$", field_value)
    if field_name == "ecl":
        return field_value in ["amb", "blu", "brn", "gry", "grn", "hzl", "oth"]
    if field_name == "pid":
        return re.match(r"^\d{9}$", field_value)
    
    # print("Defaulting to valid field for field {} with value {}!".format(field_name, field_value))
    return True

In [124]:
# check the example fields with the new validation function
assert validate_field("byr", "2002")
assert not validate_field("byr", "2003")

assert validate_field("hgt", "60in")
assert validate_field("hgt", "190cm")
assert not validate_field("hgt", "190in")
assert not validate_field("hgt", "190")

assert validate_field("hcl", "#123abc")
assert not validate_field("hcl", "#123abz")
assert not validate_field("hcl", "123abc")

assert validate_field("ecl", "brn")
assert not validate_field("ecl", "wat")

assert validate_field("pid", "000000001")
assert not validate_field("pid", "0123456789")

### Passport Validation

A rewrite of the passport validation function to check for required fields **and** make sure that all of the values are valid for the fields.

In [125]:
def valid_passport_2(passport):
    # list of fields that are required that haven't been found yet
    fields_left = ["byr", "iyr", "eyr", "hgt", "hcl", "ecl", "pid"]
    fields = passport.split(" ")
    
    for field in fields:
        # split at colon and get first item to get the name of the field
        field_name = field.split(":")[0]
        field_value = ":".join(field.split(":")[1:])
        # remove the field from the required fields list as we have found it
        if field_name in fields_left:
            fields_left.remove(field_name)
        
        if not validate_field(field_name, field_value):
            # print("Field {} is not valid with value {}".format(field_name, field_value))
            return False
    
    # if there are more than 0 fields left, it is not valid
    return len(fields_left) <= 0

In [126]:
# invalid passports
invalid_passports = [
    "eyr:1972 cid:100 hcl:#18171d ecl:amb hgt:170 pid:186cm iyr:2018 byr:1926",
    "iyr:2019 hcl:#602927 eyr:1967 hgt:170cm ecl:grn pid:012533040 byr:1946",
    "hcl:dab227 iyr:2012 ecl:brn hgt:182cm pid:021572410 eyr:2020 byr:1992 cid:277",
    "hgt:59cm ecl:zzz eyr:2038 hcl:74454a iyr:2023 pid:3556412378 byr:2007"
]

for p in invalid_passports:
    # print("Checking passport", p)
    assert valid_passport_2(p) == False

# print("=================================")
    
# valid passports
valid_passports = [
    "pid:087499704 hgt:74in ecl:grn iyr:2012 eyr:2030 byr:1980 hcl:#623a2f",
    "eyr:2029 ecl:blu cid:129 byr:1989 iyr:2014 pid:896056539 hcl:#a97842 hgt:165cm",
    "hcl:#888785 hgt:164cm byr:2001 iyr:2015 cid:88 pid:545766238 ecl:hzl eyr:2022",
    "iyr:2010 hgt:158cm hcl:#b6652a ecl:blu byr:1944 eyr:2021 pid:093154719"
]

for p in valid_passports:
    # print("Checking passport", p)
    assert valid_passport_2(p) == True

### Get the answer!

In [127]:
with open("input.txt", "r") as file:
    lines = file.read()

valid = count_valid(parse_passports(lines), func=valid_passport_2)
print("There are {} valid passports".format(valid))

There are 188 valid passports
